In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Image Generation from Verses\n",
    "\n",
    "This notebook demonstrates how to generate images based on verses from Truyện Kiều."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import os\n",
    "import torch\n",
    "import numpy as np\n",
    "from PIL import Image\n",
    "import matplotlib.pyplot as plt\n",
    "from IPython.display import display, HTML\n",
    "\n",
    "from src.image_generator import VerseToImageGenerator\n",
    "from src.preprocessor import KieuPreprocessor\n",
    "from src.cultural_context import CulturalContextEnhancer"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup Image Generator\n",
    "\n",
    "First, we'll initialize the image generator with Stable Diffusion."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check if GPU is available for faster generation\n",
    "device = \"cuda\" if torch.cuda.is_available() else \"cpu\"\n",
    "print(f\"Using device: {device}\")\n",
    "\n",
    "# Create output directory\n",
    "os.makedirs(\"../output/images\", exist_ok=True)\n",
    "\n",
    "try:\n",
    "    # Initialize the image generator\n",
    "    generator = VerseToImageGenerator()\n",
    "    print(\"Image generator initialized\")\n",
    "    \n",
    "    # Load cultural context enhancer\n",
    "    enhancer = CulturalContextEnhancer()\n",
    "    print(\"Cultural context enhancer loaded\")\n",
    "    \n",
    "except ImportError as e:\n",
    "    print(f\"Error: {e}\")\n",
    "    print(\"Please install the required packages: pip install diffusers transformers torch\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Load Sample Verses from Truyện Kiều\n",
    "\n",
    "We'll load some representative verses to visualize."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the poem\n",
    "preprocessor = KieuPreprocessor()\n",
    "verses = preprocessor.load_poem('../data/truyen_kieu.txt')\n",
    "\n",
    "# Select verses with rich imagery for visualization\n",
    "imagery_keywords = [\"trăng\", \"hoa\", \"liễu\", \"mây\", \"núi\", \"sông\", \"gió\"]\n",
    "\n",
    "# Find verses containing imagery\n",
    "imagery_verses = []\n",
    "for verse in verses:\n",
    "    if any(keyword in verse.lower() for keyword in imagery_keywords):\n",
    "        imagery_verses.append(verse)\n",
    "\n",
    "# Select a diverse set of sample verses\n",
    "selected_indices = np.linspace(0, len(imagery_verses)-1, 5, dtype=int)\n",
    "sample_verses = [imagery_verses[i] for i in selected_indices]\n",
    "\n",
    "print(\"Selected sample verses with rich imagery:\")\n",
    "for i, verse in enumerate(sample_verses):\n",
    "    print(f\"{i+1}. {verse}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Image Generation from Verses\n",
    "\n",
    "Now we'll generate images based on the selected verses."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def display_verse_and_image(verse, image, prompt=None):\n",
    "    \"\"\"Display a verse and its corresponding generated image\"\"\"\n",
    "    plt.figure(figsize=(10, 8))\n",
    "    \n",
    "    # Display the image\n",
    "    plt.imshow(image)\n",
    "    plt.axis('off')\n",
    "    \n",
    "    # Add the verse as a title\n",
    "    plt.title(f\"Verse: {verse}\\n\", fontsize=12)\n",
    "    \n",
    "    # Add the prompt if provided\n",
    "    if prompt:\n",
    "        plt.figtext(0.5, 0.01, f\"Prompt: {prompt}\", ha='center', fontsize=10)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "# Generate images for each sample verse\n",
    "for verse in sample_verses:\n",
    "    # Create prompt\n",
    "    prompt = generator.create_prompt(verse)\n",
    "    \n",
    "    # Generate image\n",
    "    print(f\"Generating image for: {verse}\")\n",
    "    image = generator.generate_image(verse)\n",
    "    \n",
    "    # Display verse and image\n",
    "    display_verse_and_image(verse, image, prompt)\n",
    "    \n",
    "    # Optional: save the image\n",
    "    # Save first 20 characters of verse as filename (sanitized)\n",
    "    filename = \"\".join(c if c.isalnum() else \"_\" for c in verse[:20])\n",
    "    image.save(f\"../output/images/{filename}.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Interactive Image Generation\n",
    "\n",
    "Let's create an interactive tool for generating images from any verse."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from ipywidgets import widgets, interact, fixed\n",
    "\n",
    "def generate_image_interactive(verse, style):\n",
    "    print(f\"Generating image for verse: {verse}\")\n",
    "    print(f\"Using style: {style}\")\n",
    "    \n",
    "    # Enhance the prompt with cultural context\n",
    "    enhanced_verse = enhancer.enhance_text_prompt(verse, \"image\")\n",
    "    \n",
    "    # Extract imagery terms\n",
    "    imagery = generator.extract_imagery(verse)\n",
    "    if imagery:\n",
    "        print(f\"Detected imagery: {', '.join(imagery)}\")\n",
    "    \n",
    "    # Generate image\n",
    "    image = generator.generate_image(verse, style=style)\n",
    "    \n",
    "    # Create prompt for display\n",
    "    prompt = generator.create_prompt(verse)\n",
    "    prompt = generator.enhance_prompt_with_style(prompt, style)\n",
    "    \n",
    "    # Display verse and image\n",
    "    display_verse_and_image(verse, image, prompt)\n",
    "    \n",
    "    return image\n",
    "\n",
    "# Create interactive widgets\n",
    "verse_widget = widgets.Textarea(\n",
    "    value=\"Trăm năm trong cõi người ta,\",\n",
    "    description=\"Verse:\",\n",
    "    layout=widgets.Layout(width='80%', height='60px')\n",
    ")\n",
    "\n",
    "style_widget = widgets.Dropdown(\n",
    "    options=['traditional', 'modern', 'fantasy', 'ukiyo-e', 'impressionist'],\n",
    "    value='traditional',\n",
    "    description='Art Style:'\n",
    ")\n",
    "\n",
    "interact(generate_image_interactive, verse=verse_widget, style=style_widget)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Generating Multiple Style Variations\n",
    "\n",
    "Let's generate different artistic style variations for the same verse."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_style_variations(verse):\n",
    "    styles = ['traditional', 'modern', 'fantasy', 'ukiyo-e', 'impressionist']\n",
    "    \n",
    "    print(f\"Generating style variations for: {verse}\")\n",
    "    \n",
    "    # Create a figure with subplots\n",
    "    fig, axes = plt.subplots(1, len(styles), figsize=(20, 5))\n",
    "    \n",
    "    for i, style in enumerate(styles):\n",
    "        # Generate image with this style\n",
    "        image = generator.generate_image(verse, style=style)\n",
    "        \n",
    "        # Display in the corresponding subplot\n",
    "        axes[i].imshow(image)\n",
    "        axes[i].set_title(f\"Style: {style.title()}\")\n",
    "        axes[i].axis('off')\n",
    "    \n",
    "    plt.suptitle(f\"Verse: {verse}\", fontsize=14)\n",
    "    plt.tight_layout(rect=[0, 0, 1, 0.95])\n",
    "    plt.show()\n",
    "\n",
    "# Choose one verse for style variations\n",
    "style_verse = \"Vầng trăng vằng vặc giữa trời,\"\n",
    "generate_style_variations(style_verse)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Generating Image Variations for a Single Verse\n",
    "\n",
    "Let's generate several variations of the same verse to see different interpretations."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_image_variations(verse, num_variations=3):\n",
    "    print(f\"Generating {num_variations} variations of: {verse}\")\n",
    "    \n",
    "    # Generate variations\n",
    "    variations = generator.generate_variations(verse, num_variations=num_variations)\n",
    "    \n",
    "    # Create a figure with subplots\n",
    "    fig, axes = plt.subplots(1, num_variations, figsize=(5*num_variations, 5))\n",
    "    \n",
    "    for i, image in enumerate(variations):\n",
    "        # Display in the corresponding subplot\n",
    "        axes[i].imshow(image)\n",
    "        axes[i].set_title(f\"Variation {i+1}\")\n",
    "        axes[i].axis('off')\n",
    "    \n",
    "    plt.suptitle(f\"Verse: {verse}\", fontsize=14)\n",
    "    plt.tight_layout(rect=[0, 0, 1, 0.95])\n",
    "    plt.show()\n",
    "\n",
    "# Choose a verse with rich visual details\n",
    "variation_verse = \"Hoa trôi lạc bước giang hồ,\"\n",
    "generate_image_variations(variation_verse)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Cultural Context Enhancement\n",
    "\n",
    "Let's use the cultural context enhancer to improve our image generation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_with_cultural_context(verse, character_or_symbol):\n",
    "    # Get contextual information\n",
    "    context_info = enhancer.get_contextual_info(character_or_symbol)\n",
    "    \n",
    "    if context_info:\n",
    "        # Print context information\n",
    "        print(f\"Adding context for: {context_info['term']}\")\n",
    "        print(f\"Category: {context_info['category']}\")\n",
    "        print(f\"Description: {context_info['info']}\")\n",
    "        \n",
    "        # Modify the verse to include the cultural element if not already there\n",
    "        if context_info['term'] not in verse.lower():\n",
    "            enhanced_verse = f\"{verse} {context_info['term']}\"\n",
    "        else:\n",
    "            enhanced_verse = verse\n",
    "            \n",
    "        # Create a culturally enhanced prompt\n",
    "        base_prompt = generator.create_prompt(enhanced_verse)\n",
    "        if context_info['category'] == 'characters':\n",
    "            enhanced_prompt = f\"{base_prompt} Featuring {context_info['term']} ({context_info['info']})\"\n",
    "        elif context_info['category'] == 'symbols':\n",
    "            enhanced_prompt = f\"{base_prompt} Prominently displaying {context_info['term']} symbolizing {context_info['info']}\"\n",
    "        else:\n",
    "            enhanced_prompt = base_prompt\n",
    "        \n",
    "        # Generate both standard and enhanced images for comparison\n",
    "        print(\"Generating standard image...\")\n",
    "        standard_image = generator.generate_image(verse)\n",
    "        \n",
    "        print(\"Generating culturally enhanced image...\")\n",
    "        # For the enhanced image, manually set the prompt\n",
    "        enhanced_image = generator.generate_image(enhanced_verse)\n",
    "        \n",
    "        # Display both images for comparison\n",
    "        fig, axes = plt.subplots(1, 2, figsize=(15, 7))\n",
    "        \n",
    "        axes[0].imshow(standard_image)\n",
    "        axes[0].set_title(\"Standard Generation\")\n",
    "        axes[0].axis('off')\n",
    "        \n",
    "        axes[1].imshow(enhanced_image)\n",
    "        axes[1].set_title(\"With Cultural Context\")\n",
    "        axes[1].axis('off')\n",
    "        \n",
    "        plt.suptitle(f\"Verse: {verse}\\nContext: {context_info['term']} - {context_info['info']}\", fontsize=12)\n",
    "        plt.tight_layout(rect=[0, 0, 1, 0.95])\n",
    "        plt.show()\n",
    "    else:\n",
    "        print(f\"No cultural context found for '{character_or_symbol}'\")\n",
    "        # Generate regular image\n",
    "        image = generator.generate_image(verse)\n",
    "        display_verse_and_image(verse, image)\n",
    "\n",
    "# Choose a verse and a cultural element to enhance\n",
    "context_verse = \"Trăm năm trong cõi người ta,\"\n",
    "cultural_element = \"kiều\"  # Main character\n",
    "\n",
    "generate_with_cultural_context(context_verse, cultural_element)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Conclusion\n",
    "\n",
    "In this notebook, we've demonstrated how to generate images based on verses from Truyện Kiều using Stable Diffusion. We've shown:\n",
    "\n",
    "1. Basic image generation from verses\n",
    "2. Interactive image generation\n",
    "3. Different artistic style variations\n",
    "4. Multiple interpretations of the same verse\n",
    "5. Enhancement using cultural context\n",
    "\n",
    "The quality of the generated images depends on the model and the descriptiveness of the prompts. The cultural context enhancer helps create more authentic visualizations by incorporating Vietnamese cultural elements."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}